In [12]:
import os, glob
#import gdal

d = r"D:\Benelux_Sentinel2"
safe_dirs = glob.glob(d + os.sep + "*.SAFE")

ndvi_process = True

print(len(safe_dirs))

113


In [13]:
granules = []
for i in safe_dirs:
    granules.append(glob.glob(i + os.sep + "GRANULE" + os.sep + "*")[0])

print(len(granules))

113


In [14]:
img_data = {}

for i in granules:
    print(i)
    img_dir = i + os.sep + "IMG_DATA"
    red = glob.glob(img_dir + os.sep + "*_B04.jp2")[0]
    green = glob.glob(img_dir + os.sep + "*_B03.jp2")[0]
    blue = glob.glob(img_dir + os.sep + "*_B02.jp2")[0]
    infrared = glob.glob(img_dir + os.sep + "*_B08.jp2")[0]
    tile = red.split(os.sep)[-1].split("_")[0]
    date = red.split(os.sep)[-1].split("_")[1].split("T")[0]
    orbit = red.split(d + os.sep)[-1].split(os.sep)[0].split("_")[4]
    satellite = red.split(d + os.sep)[-1].split(os.sep)[0].split("_")[0]
    img_name = f"{tile}_{orbit}_{satellite}_{date}"
    img_data[img_name] = {"red": red, "green": green, "blue": blue, "infrared": infrared}
print(len(list(img_data.keys())))
print(img_data)

D:\Benelux_Sentinel2\S2A_MSIL1C_20210601T104021_N0300_R008_T31UEQ_20210601T125922.SAFE\GRANULE\L1C_T31UEQ_A031036_20210601T104302
D:\Benelux_Sentinel2\S2A_MSIL1C_20210601T104021_N0300_R008_T31UER_20210601T125922.SAFE\GRANULE\L1C_T31UER_A031036_20210601T104302
D:\Benelux_Sentinel2\S2A_MSIL1C_20210601T104021_N0300_R008_T31UES_20210601T125922.SAFE\GRANULE\L1C_T31UES_A031036_20210601T104302
D:\Benelux_Sentinel2\S2A_MSIL1C_20210601T104021_N0300_R008_T31UFQ_20210601T125922.SAFE\GRANULE\L1C_T31UFQ_A031036_20210601T104302
D:\Benelux_Sentinel2\S2A_MSIL1C_20210601T104021_N0300_R008_T31UFR_20210601T125922.SAFE\GRANULE\L1C_T31UFR_A031036_20210601T104302
D:\Benelux_Sentinel2\S2A_MSIL1C_20210601T104021_N0300_R008_T31UFS_20210601T125922.SAFE\GRANULE\L1C_T31UFS_A031036_20210601T104302
D:\Benelux_Sentinel2\S2A_MSIL1C_20210601T104021_N0300_R008_T31UFT_20210601T125922.SAFE\GRANULE\L1C_T31UFT_A031036_20210601T104302
D:\Benelux_Sentinel2\S2A_MSIL1C_20210601T104021_N0300_R008_T31UFU_20210601T125922.SAFE\GRA

D:\Benelux_Sentinel2\S2B_MSIL1C_20210530T104619_N0300_R051_T31UFU_20210530T115735.SAFE\GRANULE\L1C_T31UFU_A022099_20210530T104621
D:\Benelux_Sentinel2\S2B_MSIL1C_20210530T104619_N0300_R051_T31UFV_20210530T115735.SAFE\GRANULE\L1C_T31UFV_A022099_20210530T104621
D:\Benelux_Sentinel2\S2B_MSIL1C_20210530T104619_N0300_R051_T31UGS_20210530T115735.SAFE\GRANULE\L1C_T31UGS_A022099_20210530T104621
D:\Benelux_Sentinel2\S2B_MSIL1C_20210530T104619_N0300_R051_T31UGT_20210530T115735.SAFE\GRANULE\L1C_T31UGT_A022099_20210530T104621
D:\Benelux_Sentinel2\S2B_MSIL1C_20210530T104619_N0300_R051_T31UGU_20210530T115735.SAFE\GRANULE\L1C_T31UGU_A022099_20210530T104621
D:\Benelux_Sentinel2\S2B_MSIL1C_20210530T104619_N0300_R051_T31UGV_20210530T115735.SAFE\GRANULE\L1C_T31UGV_A022099_20210530T104621
D:\Benelux_Sentinel2\S2B_MSIL1C_20210530T104619_N0300_R051_T32ULB_20210530T115735.SAFE\GRANULE\L1C_T32ULB_A022099_20210530T104621
D:\Benelux_Sentinel2\S2B_MSIL1C_20210530T104619_N0300_R051_T32ULC_20210530T115735.SAFE\GRA

In [15]:
import subprocess
import rasterio
import numpy

In [16]:
for i in list(img_data.keys()):
    file_list = [img_data[i]["red"], img_data[i]["green"], img_data[i]["blue"], img_data[i]["infrared"]]
    with rasterio.open(file_list[0]) as src0:
        meta = src0.meta
    meta.update(count = len(file_list))
    out_file_name = d + os.sep + i + ".jp2"
    if ndvi_process == True:
        band_infrared, band_red, ndvi_kwargs = rasterio.open(img_data[i]["infrared"]).read(1), rasterio.open(img_data[i]["red"]).read(1), rasterio.open(img_data[i]["infrared"]).meta
        numpy.seterr(divide='ignore', invalid='ignore')
        ndvi = ((band_infrared.astype(float) - band_red.astype(float)) / (band_infrared + band_red)) * (2**15)
        ndvi = numpy.floor(ndvi)
        ndvi = ndvi.astype(int)
        ndvi_kwargs.update(dtype=rasterio.int16, count=1)
        #print(ndvi_kwargs)
        ndvi_file_name = out_file_name.replace(".jp2", "_ndvi.jp2")
        print(ndvi_file_name)
        if os.path.exists(ndvi_file_name) == False:
            with rasterio.open(ndvi_file_name, 'w', **ndvi_kwargs) as dst:
                dst.write_band(1, ndvi.astype(rasterio.int16))
    print(out_file_name)
    meta.update(count = len(file_list))
    #print(meta)
    if os.path.exists(out_file_name) == False:
        with rasterio.open(out_file_name, "w", **meta) as dst:
            for id, layer in enumerate(file_list, start=1):
                with rasterio.open(layer) as src1:
                    dst.write_band(id, src1.read(1))

D:\Benelux_Sentinel2\T31UEQ_R008_S2A_20210601_ndvi.jp2
D:\Benelux_Sentinel2\T31UEQ_R008_S2A_20210601.jp2
D:\Benelux_Sentinel2\T31UER_R008_S2A_20210601_ndvi.jp2
D:\Benelux_Sentinel2\T31UER_R008_S2A_20210601.jp2
D:\Benelux_Sentinel2\T31UES_R008_S2A_20210601_ndvi.jp2
D:\Benelux_Sentinel2\T31UES_R008_S2A_20210601.jp2
D:\Benelux_Sentinel2\T31UFQ_R008_S2A_20210601_ndvi.jp2
D:\Benelux_Sentinel2\T31UFQ_R008_S2A_20210601.jp2
D:\Benelux_Sentinel2\T31UFR_R008_S2A_20210601_ndvi.jp2
D:\Benelux_Sentinel2\T31UFR_R008_S2A_20210601.jp2
D:\Benelux_Sentinel2\T31UFS_R008_S2A_20210601_ndvi.jp2
D:\Benelux_Sentinel2\T31UFS_R008_S2A_20210601.jp2
D:\Benelux_Sentinel2\T31UFT_R008_S2A_20210601_ndvi.jp2
D:\Benelux_Sentinel2\T31UFT_R008_S2A_20210601.jp2
D:\Benelux_Sentinel2\T31UFU_R008_S2A_20210601_ndvi.jp2
D:\Benelux_Sentinel2\T31UFU_R008_S2A_20210601.jp2
D:\Benelux_Sentinel2\T31UFV_R008_S2A_20210601_ndvi.jp2
D:\Benelux_Sentinel2\T31UFV_R008_S2A_20210601.jp2
D:\Benelux_Sentinel2\T31UGQ_R008_S2A_20210601_ndvi.jp2


D:\Benelux_Sentinel2\T31UDS_R051_S2B_20210530_ndvi.jp2
D:\Benelux_Sentinel2\T31UDS_R051_S2B_20210530.jp2
D:\Benelux_Sentinel2\T31UES_R051_S2B_20210530_ndvi.jp2
D:\Benelux_Sentinel2\T31UES_R051_S2B_20210530.jp2
D:\Benelux_Sentinel2\T31UET_R051_S2B_20210530_ndvi.jp2
D:\Benelux_Sentinel2\T31UET_R051_S2B_20210530.jp2
D:\Benelux_Sentinel2\T31UEU_R051_S2B_20210530_ndvi.jp2
D:\Benelux_Sentinel2\T31UEU_R051_S2B_20210530.jp2
D:\Benelux_Sentinel2\T31UFS_R051_S2B_20210530_ndvi.jp2
D:\Benelux_Sentinel2\T31UFS_R051_S2B_20210530.jp2
D:\Benelux_Sentinel2\T31UFT_R051_S2B_20210530_ndvi.jp2
D:\Benelux_Sentinel2\T31UFT_R051_S2B_20210530.jp2
D:\Benelux_Sentinel2\T31UFU_R051_S2B_20210530_ndvi.jp2
D:\Benelux_Sentinel2\T31UFU_R051_S2B_20210530.jp2
D:\Benelux_Sentinel2\T31UFV_R051_S2B_20210530_ndvi.jp2
D:\Benelux_Sentinel2\T31UFV_R051_S2B_20210530.jp2
D:\Benelux_Sentinel2\T31UGS_R051_S2B_20210530_ndvi.jp2
D:\Benelux_Sentinel2\T31UGS_R051_S2B_20210530.jp2
D:\Benelux_Sentinel2\T31UGT_R051_S2B_20210530_ndvi.jp2
